In [1]:
import yaml
import logging

In [2]:
# Initialize the logger once as the application starts up.
with open("../logging.yaml", 'rt') as f:
    config = yaml.safe_load(f.read())
    logging.config.dictConfig(config)
 
# Get an instance of the logger and use it to write a log!
# Note: Do this AFTER the config is loaded above or it won't use the config.
logger = logging.getLogger("py-clean-arch")
logger.info("Configured the logger!")

2023-01-09 09:31:02,421 - py-clean-arch - INFO - Configured the logger!


In [3]:
def stop_database():
   logging.info("stopping db") 
def start_database():
    logging.info("starting db")

class DBHandler:
    """for a context manager define.
        1 __enter__ method
        2 __exit__ method
    """
    def __enter__(self):
        stop_database()
        return self
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()

def db_backup():
    logging.info("doing backup")

def check():
    with DBHandler():
        db_backup()

In [4]:
check()

2023-01-09 09:31:02,486 - root - INFO - stopping db
2023-01-09 09:31:02,492 - root - INFO - doing backup
2023-01-09 09:31:02,495 - root - INFO - starting db


In [5]:
# rewriting the above using the context manager lib
import contextlib

@contextlib.contextmanager
def db_handler():
    try:
        stop_database()
        yield
    finally:
        start_database()


with db_handler():
    db_backup()

2023-01-09 09:31:02,526 - root - INFO - stopping db
2023-01-09 09:31:02,531 - root - INFO - doing backup
2023-01-09 09:31:02,538 - root - INFO - starting db


In [6]:
# other way of writing context manager by extending a class
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()
        return self
    
    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()



@dbhandler_decorator()
def offline_backup():
    db_backup()

In [7]:
offline_backup()

2023-01-09 09:31:02,590 - root - INFO - stopping db
2023-01-09 09:31:02,592 - root - INFO - doing backup
2023-01-09 09:31:02,595 - root - INFO - starting db


In [8]:
#if you want to access the object
with dbhandler_decorator() as handler:
    db_backup()

2023-01-09 09:31:02,622 - root - INFO - stopping db
2023-01-09 09:31:02,623 - root - INFO - doing backup
2023-01-09 09:31:02,625 - root - INFO - starting db


In [9]:
# if you need to create your own getter and setter with some functionality use properties decorator
class Coordinate:
    def __init__(self, lat: float, long: float) -> None:
        self._latitude = self._longitude = None
        self.latitude = lat
        self.longitude = long

    @property
    def latitude(self) -> float:
        return self._latitude

    @latitude.setter
    def latitude(self, lat_value: float) -> None:
        if lat_value not in range(-90, 90 + 1):
            raise ValueError(f"{lat_value} is an invalid value for latitude")
        self._latitude = lat_value

    @property
    def longitude(self) -> float:
        return self._longitude

    @longitude.setter
    def longitude(self, long_value: float) -> None:
        if long_value not in range(-180, 180 + 1):
            raise ValueError(f"{long_value} is an invalid value for longitude")
        self._longitude = long_value

    def __repr__(self) -> str:
        return f"Coordinate(lat: {self.latitude}, lon: {self.longitude})"

In [10]:
a = Coordinate(90,33)
print(a)
a.latitude = 50
print(a)

Coordinate(lat: 90, lon: 33)
Coordinate(lat: 50, lon: 33)
